In [85]:
# importlarımızı yapıyoruz

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler

In [86]:
# Excel dosyasının yolu
dosya_yolu = r"C:\Users\yusuf\OneDrive\Masaüstü\Laptop_Price_Prediction\cleaned_data.xlsx"

# Dosya yolunun var olup olmadığını kontrol et
if os.path.exists(dosya_yolu):
    print("Dosya mevcut!")
    # Dosyayı oku
    df_ = pd.read_excel(dosya_yolu)
    df = df_.copy()
else:
    print("Dosya yolu yanlış veya dosya mevcut değil.")

Dosya mevcut!


In [87]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ekran_Boyutu         1200 non-null   object 
 1   Ekran_Karti          1200 non-null   object 
 2   GPU_Bellek_Tipi      1200 non-null   object 
 3   GPU_Gucu             1200 non-null   int64  
 4   GPU_Hafizasi         1200 non-null   int64  
 5   GPU_Tipi             1200 non-null   object 
 6   Yenileme_Hizi        1200 non-null   int64  
 7   Max_CPU_Hizi_GHz     1200 non-null   float64
 8   Panel_Tipi           1200 non-null   object 
 9   RAM                  1200 non-null   int64  
 10  RAM_Tipi             1200 non-null   object 
 11  Renk                 1200 non-null   object 
 12  SSD_Kapasitesi       1200 non-null   int64  
 13  CPU_Cekirdek_Sayisi  1200 non-null   int64  
 14  Isletim_Sistemi      1200 non-null   object 
 15  Islemci              1200 non-null   o

In [88]:
df["Fiyat"]

0        34.999
1        34.899
2        28.089
3        34.999
4        26.499
         ...   
1195    121.480
1196     58.099
1197     58.099
1198     58.099
1199     58.099
Name: Fiyat, Length: 1200, dtype: float64

In [89]:
df["Ekran_Boyutu"].unique()

array(['15,6', '17,3', '16,1', '14', '13,3'], dtype=object)

In [90]:
# 1. Adım: Virgülleri noktaya çevir
df["Ekran_Boyutu"] = df["Ekran_Boyutu"].str.replace(',', '.', regex=False)

# 2. Adım: Sayısal değere çevir
df["Ekran_Boyutu"] = pd.to_numeric(df["Ekran_Boyutu"])

# 3. Adım: Normalizasyon
scaler = MinMaxScaler()
df["Ekran_Boyutu"] = scaler.fit_transform(df[["Ekran_Boyutu"]])

# Normalizasyon sonuçları:
# '15.6' → 0.575
# '17.3' → 1.0
# '16.1' → 0.7
# '14.0' → 0.175
# '13.3' → 0.0

# 4. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [91]:
df["Ekran_Karti"].unique()

array(['Nvidia GeForce RTX 4060', 'Nvidia GeForce RTX 3050',
       'Nvidia GeForce RTX 2050', 'Nvidia GeForce RTX 4050',
       'AMD Radeon RX 6550M', 'AMD Radeon Graphics',
       'Nvidia GeForce RTX 4070', 'Nvidia GeForce RTX 4090',
       'Nvidia GeForce GTX 1650', 'Nvidia GeForce RTX 3050 Ti',
       'Nvidia GeForce RTX 4080', 'Nvidia GeForce RTX 3070',
       'AMD Radeon RX7600S', 'Nvidia GeForce RTX 3070Ti',
       'Nvidia GeForce RTX 3080', 'Dahili Ekran Kartı',
       'Nvidia Quadro T2000', 'Nvidia GeForce MX550',
       'Nvidia GeForce GTX 1050', 'Nvidia GeForce RTX 2060',
       'Nvidia GeForce RTX3060', 'RTX 2070 SUPER Max-q',
       'AMD Radeon RX6500M', 'Nvidia GeForce GTX1650 Ti',
       'Nvidia GeForce MX330'], dtype=object)

In [92]:
# 1. Adım: Ekran kartı isimlerini normalize etme
def normalize_gpu_name(name):
    # Genel temizleme: fazla boşlukları sil, her kelimeyi büyük harfe çevir
    name = ' '.join(name.split())
    
    # "Dahili Ekran Kartı" için özel bir dönüşüm
    if 'Dahili Ekran Kartı' in name:
        return 'Integrated Graphics'
    
    # Ekran kartı serisini düzenleme (RTX, GTX, Quadro, MX gibi)
    name = re.sub(r'(?<=\d)(Ti|Super|Max-q|S|M|m)?', '', name)  # "Ti", "Super", "Max-q" gibi son ekleri temizle
    name = re.sub(r'\s{2,}', ' ', name)  # Çift boşlukları tek boşluğa dönüştür
    
    # Ekran kartı markası ve serisini doğru şekilde yazma
    name = re.sub(r'Nvidia', 'Nvidia GeForce', name)
    name = re.sub(r'GeForce\s+GeForce', 'GeForce', name)
    name = re.sub(r'RX', 'Radeon RX', name)
    
    # Sonuç olarak, temizlenmiş ve normalize edilmiş ismi döndürüyoruz
    return name.strip()

# Ekran kartı isimlerini normalize etme
df['Ekran_Karti'] = df['Ekran_Karti'].apply(normalize_gpu_name)

# 2. Adım: Etiketleme (Label Encoding) – Ekran kartı isimlerini sayısal değerlere çevirme
label_encoder = LabelEncoder()
df['Ekran_Karti'] = label_encoder.fit_transform(df['Ekran_Karti'])

# 3. Adım: Sayısal veriye dönüştürme ve normalizasyon (Min-Max Scaling)
# Bu adımda, ekran kartı isimlerini etiketledikten sonra bu değerlerin üzerinde normalizasyon yapmayacağız.
# Ancak, burada örnek olarak ekran kartı boyutları gibi sayısal veriler üzerine normalizasyon yapılabilir.
# Örneğin:
df['Ekran_Karti'] = MinMaxScaler().fit_transform(df[['Ekran_Karti']])


# 4. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [93]:
# 1. Adım: GPU bellek tipi isimlerini normalize etme
def normalize_gpu_memory_type(memory_type):
    # Sayısal bir değer gelirse, onu string'e çevir
    if isinstance(memory_type, (int, float)):
        memory_type = str(memory_type)
    
    # Genel temizleme: fazlalık boşlukları temizleme
    memory_type = ' '.join(memory_type.split())
    
    # Bellek tiplerini standart hale getirme (herhangi bir gereksiz ekleme varsa düzenlenebilir)
    memory_type = re.sub(r'(?i)gddr5|gddr6|lpddr4', lambda match: match.group(0).upper(), memory_type)

    return memory_type.strip()

# 2. Adım: Bellek tiplerini normalize etme
df['GPU_Bellek_Tipi'] = df['GPU_Bellek_Tipi'].apply(normalize_gpu_memory_type)

# 3. Adım: Etiketleme (Label Encoding) – GPU bellek tipi isimlerini sayısal değerlere çevirme
label_encoder = LabelEncoder()
df['GPU_Bellek_Tipi'] = label_encoder.fit_transform(df['GPU_Bellek_Tipi'])

# 4. Adım: Normalizasyon (Min-Max Scaling) – GPU bellek tipi için sayısal veriye dönüştürme ve normalleştirme
df['GPU_Bellek_Tipi'] = MinMaxScaler().fit_transform(df[['GPU_Bellek_Tipi']])

# 5. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [94]:
df["GPU_Bellek_Tipi"].unique()

array([0.75, 0.  , 0.25, 1.  , 0.5 ])

In [95]:
# 1. Adım: GPU tipi isimlerini normalize etme
def normalize_gpu_type(gpu_type):
    # Genel temizleme: fazlalık boşlukları temizleme ve her kelimeyi büyük harfe çevirme
    gpu_type = ' '.join(gpu_type.split())
    
    # GPU tiplerini standart hale getirme (Örnek olarak)
    gpu_type = gpu_type.lower()  # Tüm isimleri küçük harfe çeviriyoruz
    gpu_type = gpu_type.capitalize()  # Sadece ilk harfi büyük yapıyoruz

    return gpu_type.strip()

# 2. Adım: GPU tipi isimlerini normalize etme
df['GPU_Tipi'] = df['GPU_Tipi'].apply(normalize_gpu_type)

# 3. Adım: Etiketleme (Label Encoding) – GPU tipi isimlerini sayısal değerlere çevirme
label_encoder = LabelEncoder()
df['GPU_Tipi'] = label_encoder.fit_transform(df['GPU_Tipi'])

# 4. Adım: Normalizasyon (Min-Max Scaling) – GPU tipi için sayısal veriye dönüştürme ve normalleştirme
df['GPU_Tipi'] = MinMaxScaler().fit_transform(df[['GPU_Tipi']])

# 5. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [96]:
# 1. Adım: Panel tipi isimlerini normalize etme
def normalize_panel_type(panel_type):
    # Genel temizleme: fazlalık boşlukları temizleme ve her kelimeyi büyük harfe çevirme
    panel_type = ' '.join(panel_type.split())
    
    # Panel tiplerini standart hale getirme (Örnek olarak)
    panel_type = panel_type.lower()  # Tüm isimleri küçük harfe çeviriyoruz
    panel_type = panel_type.capitalize()  # Sadece ilk harfi büyük yapıyoruz

    return panel_type.strip()

# 2. Adım: Panel tipi isimlerini normalize etme
df['Panel_Tipi'] = df['Panel_Tipi'].apply(normalize_panel_type)

# 3. Adım: Etiketleme (Label Encoding) – Panel tipi isimlerini sayısal değerlere çevirme
label_encoder = LabelEncoder()
df['Panel_Tipi'] = label_encoder.fit_transform(df['Panel_Tipi'])

# 4. Adım: Normalizasyon (Min-Max Scaling) – Panel tipi için sayısal veriye dönüştürme ve normalleştirme
df['Panel_Tipi'] = MinMaxScaler().fit_transform(df[['Panel_Tipi']])

# 5. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [97]:
# 1. Adım: RAM tipi isimlerini normalize etme
def normalize_ram_type(ram_type):
    # Genel temizleme: fazlalık boşlukları temizleme ve her kelimeyi büyük harfe çevirme
    ram_type = ' '.join(ram_type.split())
    
    # RAM tiplerini standart hale getirme (Örnek olarak)
    ram_type = ram_type.lower()  # Tüm isimleri küçük harfe çeviriyoruz
    ram_type = ram_type.capitalize()  # Sadece ilk harfi büyük yapıyoruz

    return ram_type.strip()

# 2. Adım: RAM tipi isimlerini normalize etme
df['RAM_Tipi'] = df['RAM_Tipi'].apply(normalize_ram_type)

# 3. Adım: Etiketleme (Label Encoding) – RAM tipi isimlerini sayısal değerlere çevirme
label_encoder = LabelEncoder()
df['RAM_Tipi'] = label_encoder.fit_transform(df['RAM_Tipi'])

# 4. Adım: Normalizasyon (Min-Max Scaling) – RAM tipi için sayısal veriye dönüştürme ve normalleştirme
df['RAM_Tipi'] = MinMaxScaler().fit_transform(df[['RAM_Tipi']])

# 5. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [98]:
# 1. Adım: Renk isimlerini normalize etme
def normalize_color(color):
    # Genel temizleme: fazlalık boşlukları temizleme ve her kelimeyi büyük harfe çevirme
    color = ' '.join(color.split())
    
    # Renk isimlerini standart hale getirme (örneğin, küçük harfe dönüştürme)
    color = color.lower()  # Tüm isimleri küçük harfe çeviriyoruz
    color = color.capitalize()  # Sadece ilk harfi büyük yapıyoruz

    return color.strip()

# 2. Adım: Renk isimlerini normalize etme
df['Renk'] = df['Renk'].apply(normalize_color)

# 3. Adım: Etiketleme (Label Encoding) – Renk isimlerini sayısal değerlere çevirme
label_encoder = LabelEncoder()
df['Renk'] = label_encoder.fit_transform(df['Renk'])

# 4. Adım: Normalizasyon (Min-Max Scaling) – Renk için sayısal veriye dönüştürme ve normalleştirme
df['Renk'] = MinMaxScaler().fit_transform(df[['Renk']])

# 5. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [99]:
# 1. Adım: İşletim sistemi isimlerini normalize etme
def normalize_os(os_name):
    # Genel temizleme: fazlalık boşlukları temizleme ve her kelimeyi büyük harfe çevirme
    os_name = ' '.join(os_name.split())
    
    # İşletim sistemi isimlerini standart hale getirme (Örnek olarak)
    os_name = os_name.lower()  # Tüm isimleri küçük harfe çeviriyoruz
    os_name = os_name.capitalize()  # Sadece ilk harfi büyük yapıyoruz

    return os_name.strip()

# 2. Adım: İşletim sistemi isimlerini normalize etme
df['Isletim_Sistemi'] = df['Isletim_Sistemi'].apply(normalize_os)

# 3. Adım: Etiketleme (Label Encoding) – İşletim sistemi isimlerini sayısal verilere çevirme
label_encoder = LabelEncoder()
df['Isletim_Sistemi'] = label_encoder.fit_transform(df['Isletim_Sistemi'])

# 4. Adım: Normalizasyon (Min-Max Scaling) – İşletim sistemi için sayısal veriye dönüştürme ve normalleştirme
df['Isletim_Sistemi'] = MinMaxScaler().fit_transform(df[['Isletim_Sistemi']])

# 5. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx


In [100]:
# 1. Adım: İşlemci isimlerini normalize etme
def normalize_processor(processor_name):
    # Genel temizleme: fazlalık boşlukları temizleme ve her kelimeyi büyük harfe çevirme
    processor_name = ' '.join(processor_name.split())
    
    # İşlemci isimlerini küçük harfe çevirme (İçindeki harfleri normalize etme)
    processor_name = processor_name.lower()  # Tüm isimleri küçük harfe çeviriyoruz
    processor_name = processor_name.capitalize()  # Sadece ilk harfi büyük yapıyoruz

    return processor_name.strip()

# 2. Adım: İşlemci isimlerini normalize etme
df['Islemci'] = df['Islemci'].apply(normalize_processor)

# 3. Adım: Etiketleme (Label Encoding) – İşlemci isimlerini sayısal verilere çevirme
label_encoder = LabelEncoder()
df['Islemci'] = label_encoder.fit_transform(df['Islemci'])

# 4. Adım: Normalizasyon (Min-Max Scaling) – İşlemci için sayısal veriye dönüştürme ve normalleştirme
df['Islemci'] = MinMaxScaler().fit_transform(df[['Islemci']])

# 5. Adım: Excel dosyasına kaydetme
df.to_excel('cleaned_data_modified.xlsx', index=False)

print("Dosya kaydedildi: cleaned_data_modified.xlsx")

Dosya kaydedildi: cleaned_data_modified.xlsx
